## 3. Task retries

Let's consider two types of exceptions:
1. **system errors** (e.g., Python-level exceptions)
2. **application-level errors** (e.g., a machine fails)

Ray will automatically **retry a task up to 3 times**, if it fails due to a system error (e.g., a worker node dies).

Below task won't be retried by default because it's an application failure

In [ ]:
@ray.remote
def incorrect_square(x: int, prob: float) -> int:
    # Simulate potential failures
    if random.random() < prob:  # % chance of failure
        raise ValueError("Random failure")
    return x**2

In [ ]:
try:
    ray.get([incorrect_square.remote(x=4, prob=0.5) for _ in range(10)])
except ray.exceptions.RayTaskError:
    print("At least one of the tasks failed", flush=True)

Ray let's you specify how to handle retries when an exception is encountered.

Let's retry on `ValueError`, like below:

In [ ]:
@ray.remote(retry_exceptions=[ValueError])
def correct_square(x: int, prob: float) -> int:
    # Simulate potential failures
    if random.random() < prob:  # % chance of failure
        raise ValueError("Random failure")

    return x**2

Note we did not have to re-define the remote function, instead we could have used `.options`

In [ ]:
correct_square_mod = correct_square.options(
    retry_exceptions=[ValueError], max_retries=10,
)

Let's try it out:

In [ ]:
try:
    outputs = ray.get([correct_square_mod.remote(x=4, prob=0.5) for _ in range(10)])
except ray.exceptions.RayTaskError:
    print("At least one of the tasks failed", flush=True)

outputs

<div class="alert alert-info">
Refer to the <strong><a href="https://docs.ray.io/en/latest/ray-core/tasks/retries.html" target="_blank">retries</a></strong> to learn more.
</div>